# Code for detecting lanes in Video


In [30]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
%matplotlib inline
import os
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML


### Global Variables

In [31]:
p_left_lane = []
p_right_lane = []

### Driver Code
This code takes input video file and writes an output video file with lanes highlighted in red color

In [35]:
input_file = "challenge.mp4"
output_file = 'challenge-out.mp4'
clip1 = VideoFileClip(input_file)
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(output_file, audio=False)

[MoviePy] >>>> Building video challenge-out.mp4
[MoviePy] Writing video challenge-out.mp4




  0%|          | 0/251 [00:00<?, ?it/s]

  1%|          | 2/251 [00:00<00:15, 16.31it/s]

  2%|▏         | 5/251 [00:00<00:14, 17.35it/s]

  3%|▎         | 8/251 [00:00<00:13, 18.40it/s]

  4%|▍         | 11/251 [00:00<00:12, 19.35it/s]

  6%|▌         | 14/251 [00:00<00:11, 19.98it/s]

  7%|▋         | 17/251 [00:00<00:11, 20.51it/s]

  8%|▊         | 20/251 [00:00<00:11, 20.73it/s]

  9%|▉         | 23/251 [00:01<00:10, 20.83it/s]

 10%|█         | 26/251 [00:01<00:10, 21.00it/s]

 12%|█▏        | 29/251 [00:01<00:10, 21.30it/s]

 13%|█▎        | 32/251 [00:01<00:10, 20.98it/s]

 14%|█▍        | 35/251 [00:01<00:10, 20.35it/s]

 15%|█▌        | 38/251 [00:01<00:11, 18.25it/s]

 16%|█▌        | 40/251 [00:02<00:11, 17.72it/s]

 17%|█▋        | 42/251 [00:02<00:11, 18.13it/s]

 18%|█▊        | 45/251 [00:02<00:10, 18.74it/s]

 19%|█▊        | 47/251 [00:02<00:10, 19.08it/s]

 20%|█▉        | 50/251 [00:02<00:10, 19.47it/s]

 21%|██        | 52/251 [00:02<00:10, 19.50it/s]

 22%|██▏  

[MoviePy] Done.
[MoviePy] >>>> Video ready: challenge-out.mp4 

CPU times: user 18 s, sys: 4 s, total: 22 s
Wall time: 15.5 s


### Process Image function
Takes images from video frames and returns processed images
Steps 
1. Get masked image via color thresholding 
2. Get masked image via canny 
3. Blend the two images
4. Perform Hough transform to get line segments
5. Get points for left and right lanes
6. Draw lane image 
7. Merge it with original image

In [33]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    #print(image.shape)
    shape = image.shape
    image = cv2.resize(image,( 960, 540), interpolation = cv2.INTER_CUBIC)

    ## get masked image via color thresholding
    masked_color_img = get_masked_color_threshold_img(image)
    ## get masked image via canny 
    canny_img = get_masked_canny_img(image)
    ## blend the two images
    blended = cv2.addWeighted(masked_color_img, 0.4, canny_img, 0.6,0)
       
    ## perform Hough transform
    lines = get_hough_lane_segments(blended)
    ## get lane points
    (left_lane, right_lane) = get_lane_points(lines)
 
        
    ## lane image
    lane_image = get_lane_image2(image,left_lane, right_lane )
    #plt.imshow(lane_image)
    
    ## merge it with the orignal image
    result = weighted_img(lane_image, image)
    result = cv2.resize(result,( shape[1], shape[0] ), interpolation = cv2.INTER_CUBIC)
    #
    #p_left_lane = np.copy(left_lane)
    #p_right_lane= np.copy(right_lane)
    return result

In [29]:
""" Uses color thresholds to return lane image. Resultant image is grayscale masked and only the lanes are shown"""
def get_masked_color_threshold_img(image) :
    ## make copy of the image
    color_img = np.copy(image)
    ## rgb threshold
    rgb_threshold = [200,200,200]

    # Identify pixels below the threshold and set them to zero
    thresholds = (image[:,:,0] < rgb_threshold[0]) | (image[:,:,1] < rgb_threshold[1]) | (image[:,:,2] < rgb_threshold[2])
    color_img[thresholds == True] = [0,0,0]


    ## apply area of interest
    img_shape= color_img.shape
    vertices = np.array([[(0,img_shape[0]),(470, 320), (470, 320), (img_shape[1],img_shape[0])]], dtype=np.int32)
    masked_color_img = region_of_interest(color_img, vertices)
    return grayscale(masked_color_img)

def get_hsv_masked_img(image):
    ## make copy of the image
    color_img = np.copy(image)

    img = cv2.cvtColor(color_img, cv2.COLOR_RGB2HSV)
    #plt.imshow(hsv, cmap='hsv')

    ## yellow lane filter
    yellow_min = np.array([10, 40, 80], np.uint8)
    yellow_max = np.array([40, 100, 140], np.uint8)
    yellow_mask = cv2.inRange(img, yellow_min, yellow_max)
    #plt.imshow(yellow_mask)
    
    ## white lane filter
    white_min = np.array([0, 0, 200], np.uint8)
    white_max = np.array([255, 80, 255], np.uint8)
    white_mask = cv2.inRange(img, white_min, white_max)

    res = cv2.bitwise_and(img, img, mask=cv2.bitwise_or(yellow_mask, white_mask))
    return res


""" Uses canny image algorithm toreturn lane image. Resultant image is grayscale masked and only the lanes are shown"""
def get_masked_canny_img(image):
    ## make copy of the image
    canny_img = np.copy(image)

    ## parameter
    guassian_blur_kernel = 5
    canny_low_threshold =50
    canny_high_threshold = 150
    img_shape= canny_img.shape
    ## convert to grayscale
    canny_img = grayscale(canny_img)
    ## appy guassian blur with
    canny_img = gaussian_blur(canny_img, guassian_blur_kernel)
    ## apply canny filter 
    canny_img = canny(canny_img, canny_low_threshold, canny_high_threshold)
    ## apply area of interest
    vertices = np.array([[(0,img_shape[0]-40),(470, 320), (470, 320), (img_shape[1],img_shape[0]-40)]], dtype=np.int32)
    canny_img = region_of_interest(canny_img, vertices)
    return canny_img

"""Apply Hough transform to get lane segments"""
def get_hough_lane_segments(image):
    rho = 2# distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 5  # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 20 #minimum number of pixels making up a line
    max_line_gap = 60   # maximum gap in pixels between connectable line segments

    lines = cv2.HoughLinesP(image, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    return lines
"""Takes input as array of HoughLine segments and returns left and right lane points arrays"""
def get_lane_points(lines):
    ldata=[]
    rdata=[]
    lslope=[] 
    rslope=[]
    first=True
    for line in lines:
         for x1,y1,x2,y2 in line:
            slope = (y2-y1)/(x2-x1)
            if ( slope < 0 ):
                ldata.extend([x1,y1,x2,y2])
                lslope.append(abs((y2-y1)/(x2-x1)))
            else: 
                rdata.extend([x1,y1])
                rslope.append(abs((y2-y1)/(x2-x1)))

    mlslope = np.mean(lslope)
    mrslope = np.mean(rslope)
    left_data = []
    right_data = []
    for line in lines : 
        for x1,y1,x2,y2 in line:
            slope = ((y2-y1)/(x2-x1)) 
       
             ## filter out horizontal lines
            if ( abs(slope) < 0.1): 
                #print ("ignoring - " ,x1,y1,x2,y2, slope)
                continue
        
            elif ( slope > 0  and abs(mrslope-abs(slope)) < 0.2):
                right_data.extend([x1,y1,x2,y2])
           
            elif ( slope < 0 and abs(mlslope-abs(slope)) < 0.2 ) : 
                left_data.extend([x1, y1,x2,y2])
            
            else:
                ## ignore this line
                #print ("ignoring - " ,x1,y1,x2,y2, slope)
                continue

            
    #print(right_data)
    ld = np.array(left_data).reshape((-1, 2))
    rd = np.array(right_data).reshape((-1, 2))
    
    
    ## starting points
    #ld = np.append(ld, np.array([180,540]).reshape((-1,2)), axis=0)
    #rd = np.append(rd, np.array([850,540]).reshape((-1,2)), axis=0)
    return ( ld, rd)

"""Draw lanes on masked image from left and right lane points and return masked lane image"""
def get_lane_image(image, ld,rd):
    ## fit lane points to a polynomial function
    left_func = np.poly1d(np.polyfit(ld[:,1], ld[:,0] , 2))
    right_func = np.poly1d(np.polyfit(rd[:,1], rd[:,0] , 2))
    
    lane_img = np.copy(image)
    lane_img[:,:,:] = [0,0,0]
    
    ## left lane line
    cv2.line(lane_img, (int(left_func(540)), 540), (int(left_func(350)), 350) , [255, 0, 0], 6)
    ## right lane line
    cv2.line(lane_img, (int(right_func(540)), 540), (int(right_func(350)), 350) , [255, 0, 0], 6)

    return lane_img


"""Draw lanes on masked image from left and right lane points and return masked lane image"""
def get_lane_image2(image, ld,rd):
    
    lane_img = np.copy(image)
    lane_img[:,:,:] = [0,0,0]
    global p_left_lane
    global p_right_lane
    y1 = 540
    y2 = 350 
    
    ## fit lane points to a polynomial function
    if ( ld.size != 0 and rd.size !=0  ): 
        left_func = np.poly1d(np.polyfit(ld[:,1], ld[:,0] , 2))
        right_func = np.poly1d(np.polyfit(rd[:,1], rd[:,0] , 2))
        ## for the first frame
        if ( not p_left_lane ): 
                p_left_lane = [left_func(y1),left_func(y2)  ]
        if ( not  p_right_lane):
                p_right_lane = [right_func(y1), right_func(y2)]
        ## reduce jittry effect
        left_x1 = int( ( 0.3 * left_func(y1)+ 0.7 * p_left_lane[0] ) /2 )
        left_x2 = int(( 0.3* left_func(y2) + 0.7 * p_left_lane[1] )/2 )
        right_x1 = int( ( 0.3 * right_func(y1)+ 0.7*   p_right_lane[0] ) /2 )
        right_x2 = int( ( 0.3 * right_func(y2)+ 0.7 *p_right_lane[1] ) /2 )
     ## Fix for when frame image has no good lane markings 
    elif(p_left_lane and p_right_lane )  : 
        left_x1 = p_left_lane[0]
        left_x2 = p_left_lane[1]
        right_x1 = p_right_lane[0]
        right_x2 = p_right_lane[1]
    else: 
       ## error in first frame
        return lane_img
    
    ## for one off case when line is bad
    if ( abs(left_x1 -p_left_lane[0] ) > 15 ) :
        left_x1 = p_left_lane[0]
    if ( abs(right_x1 -p_right_lane[0] ) > 15 ) :
        right_x1 = p_right_lane[0] 
    
    if ( abs(left_x2 -p_left_lane[1] ) > 15 ) :
        left_x2 = p_left_lane[1]
    if ( abs(right_x2 -p_right_lane[1] ) > 15 ) :
        right_x2 = p_right_lane[1] 
        
    p_left_lane = [left_x1, left_x2]
    p_right_lane = [right_x1, right_x2]
    
    ## left lane line
    cv2.line(lane_img, (int(left_x1) ,y1 ), (int(left_x2),y2 ) , [255, 0, 0], 15)
    ## right lane line
    cv2.line(lane_img, (int(right_x1), y1), (int(right_x2), y2) , [255, 0, 0], 15)

    return lane_img



## Process functions

In [24]:
## Helper Functions

In [25]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)